In [1]:
from rscube.aws import upload_file_to_s3
from pathlib import Path
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# Upload Directory

Uses *relative* path to generate prefix and key for s3 bucket. Assumes metadata.json.

Probably should re-write this so no side-effects.

In [2]:
class_directories = list(Path('classification_outputs_peckel').glob('*/'))
class_directories

[PosixPath('classification_outputs_peckel/20211011_155455_52_2262'),
 PosixPath('classification_outputs_peckel/20210925_191708_94_2408'),
 PosixPath('classification_outputs_peckel/20211014_152910_49_2459'),
 PosixPath('classification_outputs_peckel/20210926_072414_71_2233'),
 PosixPath('classification_outputs_peckel/20211003_161639_91_241d'),
 PosixPath('classification_outputs_peckel/.DS_Store'),
 PosixPath('classification_outputs_peckel/20210914_051809_10_2455'),
 PosixPath('classification_outputs_peckel/20210901_175205_71_2426'),
 PosixPath('classification_outputs_peckel/20211028_045455_02_2459'),
 PosixPath('classification_outputs_peckel/20210906_183639_52_240c'),
 PosixPath('classification_outputs_peckel/20210924_082025_48_2424'),
 PosixPath('classification_outputs_peckel/20211027_151445_21_2231'),
 PosixPath('classification_outputs_peckel/20211030_142613_41_227b'),
 PosixPath('classification_outputs_peckel/20211010_080351_47_2458'),
 PosixPath('classification_outputs_peckel/202110

In [3]:
def upload_directory(dir_path: Path) -> list:
    df_meta = gpd.GeoDataFrame.from_file(dir_path / 'metadata.geojson')
    
    BUCKET = df_meta.s3_bucket[0]
    PREFIX = df_meta.s3_prefix[0]
    assert(PREFIX == str(dir_path))
    
    paths = list(dir_path.glob('*'))
    paths = list(filter(lambda x: 'cropped' not in x.name, paths))
    
    upload_file_to_s3_p = lambda path: upload_file_to_s3(path, BUCKET, PREFIX)

    return list(map(upload_file_to_s3_p, tqdm(paths, desc=f'Uploading files in {dir_path.name}')))

In [ ]:
list(map(upload_directory, tqdm(class_directories, desc='Uploading Directories')))

Uploading files in 20211011_155455_52_2262:   0%|   | 0/7 [00:00<?, ?it/s]
Uploading files in 20211011_155455_52_2262:  14%|▏| 1/7 [00:02<00:15,  2.6
Uploading files in 20211011_155455_52_2262:  29%|▎| 2/7 [00:02<00:05,  1.1
Uploading files in 20211011_155455_52_2262:  43%|▍| 3/7 [00:03<00:03,  1.0

# Aggregate Table

In [ ]:
import boto3

s3client = boto3.client('s3')
BUCKET_NAME = 'opera-calval-dswx-jupyter'
objects = s3client.list_objects(Bucket=BUCKET_NAME)['Contents']
objects[:5]

In [ ]:
objs_meta = list(filter(lambda obj: 'metadata.geojson' == obj['Key'].split('/')[-1], objects))
objs_meta

In [ ]:
def read_one(obj):
    key = obj['Key']
    return gpd.read_file(f's3://{BUCKET_NAME}/{key}')

In [ ]:
dfs = list(map(read_one, objs_meta))
df_meta_all = pd.concat(dfs, axis=0)
df_meta_all

In [ ]:
TABLE_NAME = f'dswx_classification_rf_peckel.geojson'
df_meta_all.to_file(f's3://{BUCKET_NAME}/{TABLE_NAME}', driver='GeoJSON')